In [1]:
#@title 1. MONTAR EL DRIVE { display-mode: "form" }
import os
from google.colab import drive
drive.mount('/content/drive')
#print(os.getcwd())

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#@title Importación de libreríías { display-mode: "form" }
from tensorflow.keras import datasets, layers, models
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import cv2
import os
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
#Ruta del directorio del dataset
DatasetDir = os.path.join(os.getcwd(), "drive", "My Drive", "Proyecto_IA", "HAM10000_Data")
df = pd.read_csv(os.path.join(DatasetDir,'HAM10000_metadata.csv'))

In [4]:
def get_paths_labels(DatasetDir):
  train_metadata ='HAM10000_metadata.csv'
  test_metadata = "ISIC2018_Task3_Test_NatureMedicine_AI_Interaction_Benefit.csv"

  train = os.path.join(DatasetDir, train_metadata) 

  df = pd.read_csv(train)

  lb_style = LabelBinarizer()
  lb_results = lb_style.fit_transform(df.dx_type)
  target = pd.DataFrame(  lb_results, columns=lb_style.classes_)
  target["image_id"] = df["image_id"]

  images_path = os.path.join(DatasetDir,"HAM10000_images")
  images_pathS = os.path.join(DatasetDir,"HAM10000_segmentations")

  paths =  [images_path + "/" + id for id in target["image_id"]]
  target = [images_pathS + "/" + id for id in target["image_id"]]

  return (pd.Series(paths), pd.Series(target))
  

In [5]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=100, dim=(128,128), n_channels=3,
                 n_classes=4, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def read_image(self,path, dim=(128,128)):
      im = None
      if (os.path.exists(path+".jpg")):
        
        im = cv2.resize(cv2.imread(path+".jpg"), dim)*1./255
      else:
        print(path+".jpg")
        raise Exception("Error cargando: ", path.split("/")[-1]+".jpg" )
      return im
      
    def get_target(self,labels, dim=(128,128)):
      im = None
      seg_path = os.path.join(DatasetDir,"HAM10000_segmentations",labels+"_segmentation.png")
      if os.path.exists(seg_path):
        im =  cv2.resize(cv2.imread(seg_path), dim)[:,:,0]*1./255
      else:
        print(seg_path)
        raise Exception("Error cargando: ", seg_path.split("/")[-1])
      return im

    def __getitem__(self, index):
        
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs.iloc[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, *self.dim))
#        y = np.empty((self.batch_size,self.n_classes))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = self.read_image(ID, self.dim)
            # Store class
            y[i,] = self.get_target(ID.split("/")[-1], self.dim)

        return X, y

    def one_data(self, ID):
      return  self.read_image(ID, self.dim), self.get_target(ID,self.dim)


In [6]:
BATCH_SIZE = 32
X,Y = get_paths_labels(DatasetDir)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

training_generator = DataGenerator(X_train, Y_train, batch_size=BATCH_SIZE)
testing_generator = DataGenerator(X_test, Y_test, batch_size=BATCH_SIZE)

In [7]:
#@title U-net { display-mode: "form" }

def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c


def UNet():
    f = [16, 32, 64, 128, 256]
    image_size = 128
    inputs = keras.layers.Input((image_size, image_size, 3))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = keras.models.Model(inputs, outputs)
    return model

In [ ]:
#
log_folder = "/"
for folder in DatasetDir.split("/")[:-1]:
  log_folder = os.path.join(log_folder, folder)


tensorboard_dir = os.path.join("logs","fit_tensorboard_seg",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

checkpoint_path =  os.path.join(log_folder,"checkpoints_model_seg", "cp.ckpt")
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tensorboard_dir, histogram_freq=1)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=False,
                                                 verbose=1)

In [ ]:
os.path.exists(log_folder)

True

In [ ]:
BATCH_SIZE = 32

# Asegurarse de tener seleccionada la GPU como acelerador de hardware
with tf.device("/gpu:0"):
  model = UNet()
  model.compile(optimizer='adam',
   loss='binary_crossentropy',
   metrics=['accuracy'])

  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



  training_generator = DataGenerator(X_train, Y_train, batch_size=BATCH_SIZE)
  testing_generator = DataGenerator(X_test, Y_test, batch_size=BATCH_SIZE)

  model.fit(training_generator , epochs=50,batch_size=BATCH_SIZE , validation_data=testing_generator , callbacks=[tensorboard_callback, cp_callback])

Epoch 1/19
  1/250 [..............................] - ETA: 0s - loss: 0.6878 - accuracy: 0.7044WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
250/250 [==============================] - ETA: 0s - loss: 0.2806 - accuracy: 0.8837 
Epoch 00001: saving model to /content/drive/My Drive/Proyecto_IA/checkpoints_model_seg/cp.ckpt
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/My Drive/Proyecto_IA/checkpoints_model_seg/cp.ckpt/assets
250/250 [==============================] - 7166s 29s/step - loss: 0.2806 - accuracy: 0.8837 - val_loss: 

In [ ]:
with tf.device("/gpu:0"):

  model.fit(training_generator , epochs=31,batch_size=BATCH_SIZE , validation_data=testing_generator , callbacks=[tensorboard_callback, cp_callback])

Epoch 1/31
250/250 [==============================] - ETA: 0s - loss: 0.0558 - accuracy: 0.9744
Epoch 00001: saving model to /content/drive/My Drive/Proyecto_IA/checkpoints_model_seg/cp.ckpt
INFO:tensorflow:Assets written to: /content/drive/My Drive/Proyecto_IA/checkpoints_model_seg/cp.ckpt/assets
250/250 [==============================] - 173s 690ms/step - loss: 0.0558 - accuracy: 0.9744 - val_loss: 0.1329 - val_accuracy: 0.9572
Epoch 2/31
250/250 [==============================] - ETA: 0s - loss: 0.0521 - accuracy: 0.9759
Epoch 00002: saving model to /content/drive/My Drive/Proyecto_IA/checkpoints_model_seg/cp.ckpt
INFO:tensorflow:Assets written to: /content/drive/My Drive/Proyecto_IA/checkpoints_model_seg/cp.ckpt/assets
250/250 [==============================] - 173s 692ms/step - loss: 0.0521 - accuracy: 0.9759 - val_loss: 0.1340 - val_accuracy: 0.9569
Epoch 3/31
250/250 [==============================] - ETA: 0s - loss: 0.0524 - accuracy: 0.9758
Epoch 00003: saving model to /conten